In [1]:
import ray

In [2]:
ray.init()

2020-08-07 00:57:18,945	INFO resource_spec.py:212 -- Starting Ray with 1.42 GiB memory available for workers and up to 0.73 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-07 00:57:19,211	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-08-07 00:57:19,505	INFO services.py:1165 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '10.126.135.147',
 'raylet_ip_address': '10.126.135.147',
 'redis_address': '10.126.135.147:6379',
 'object_store_address': '/tmp/ray/session_2020-08-07_00-57-18_944430_26832/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-08-07_00-57-18_944430_26832/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-08-07_00-57-18_944430_26832'}

In [3]:
@ray.remote
def f(x):
    return x * x

In [4]:
%%time

futures = [f.remote(i) for i in range(4)]

CPU times: user 2.31 ms, sys: 950 µs, total: 3.26 ms
Wall time: 3.16 ms


In [5]:
%%time

print(ray.get(futures))

[0, 1, 4, 9]
CPU times: user 44 ms, sys: 16.6 ms, total: 60.6 ms
Wall time: 1.7 s


In [6]:
import ray

@ray.remote
class Counter(object):
    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1

    def read(self):
        return self.n

In [7]:
global e
temp = None
try:
    ray.init()
except Exception as e:
    print(e)
    temp = e

Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.


In [8]:
print(type(temp))
temp.args[0].split('?')[0]

<class 'RuntimeError'>


'Maybe you called ray.init twice by accident'

In [9]:
%%time 

counters = [Counter.remote() for i in range(4)]
[c.increment.remote() for c in counters]

CPU times: user 0 ns, sys: 6.68 ms, total: 6.68 ms
Wall time: 5.95 ms


[ObjectID(55c3b2b635949d817e0a4dfc010000c001000000),
 ObjectID(9fc77bf30b4389976f53dca1010000c001000000),
 ObjectID(9d28cb176c7f75016170691e010000c001000000),
 ObjectID(15c675b22d037e3bcd8f5689010000c001000000)]

In [10]:
%%time

futures = [c.read.remote() for c in counters]

CPU times: user 1.01 ms, sys: 0 ns, total: 1.01 ms
Wall time: 706 µs


In [11]:
%%time

print(ray.get(futures))

[1, 1, 1, 1]
CPU times: user 1.97 ms, sys: 0 ns, total: 1.97 ms
Wall time: 1.43 ms


# DE with Ray

In [12]:
import numpy as np
seed = 0
np.random.seed(0)


def eggholder(x):
    return (-(x[1] + 47) * np.sin(np.sqrt(abs(x[0]/2 + (x[1]  + 47)))) - \
            x[0] * np.sin(np.sqrt(abs(x[0] - (x[1]  + 47)))))

bounds = [(-512, 512), (-512, 512)]

In [13]:
import time

def eggholder_objective(x):
    '''Evaluates x on the eggholder function (minimizing)
    
    x: a list/array of length 2 with each value in [0, 1]
    '''
    start = time.time()
    bound_range = [bounds[0][1] - bounds[0][0]]
    bound_range.append(bounds[1][1] - bounds[1][0])
    bound_min = [bounds[0][0]]
    bound_min.append(bounds[1][0])
    x_new = x.copy()
    for i in range(len(x)):
        x_new[i] = bound_min[i] + x[i] * bound_range[i]
    
    # the fitness which determines the selection process in DE
    ## the lower the fitness value, the better is x adjudged to be
    fitness = eggholder(x_new)
    # runtime of objective, or None, or equivalent
    cost = time.time() - start 
    
    return fitness, cost

In [14]:
%%time

### introducing sleep

sleep_low = 0.2
sleep_high = 0.5

def eggholder_objective_sleep(x):
    '''Evaluates x on the eggholder function (minimizing)
    
    x: a list/array of length 2 with each value in [0, 1]
    '''
    start = time.time()
    bound_range = [bounds[0][1] - bounds[0][0]]
    bound_range.append(bounds[1][1] - bounds[1][0])
    bound_min = [bounds[0][0]]
    bound_min.append(bounds[1][0])
    x_new = x.copy()
    for i in range(len(x)):
        x_new[i] = bound_min[i] + x[i] * bound_range[i]
    
    # the fitness which determines the selection process in DE
    ## the lower the fitness value, the better is x adjudged to be
    fitness = eggholder(x_new)
    # 25% chance of a long sleep of 2 seconds to simulate bottlenecks
    if np.random.uniform() > 0.75:
        time.sleep(2)
    else:
        time.sleep(np.random.uniform(low=sleep_low, high=sleep_high))
    # runtime of objective, or None, or equivalent
    cost = time.time() - start 
    
    return fitness, cost

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 22.4 µs


In [15]:
from denas import DE, PDE #, AsyncPDE

In [16]:
# Sequential

de = DE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=5,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
)
start = time.time()
traj, runtime, hist = de.run(generations=5, verbose=True)
end = time.time() - start
    
print("Time taken for sequential processing: {} seconds".format(end))
print(de.inc_config, de.inc_score)

Initializing and evaluating new population...
Running evolutionary search...
Generation 1 /5  -- -486.1692
Generation 2 /5  -- -555.9821
Generation 3 /5  -- -555.9821
Generation 4 /5  -- -555.9821
Generation 5 /5  -- -555.9821

Run complete!
Time taken for sequential processing: 22.874417781829834 seconds
[0.96366276 0.92642063] -555.9820772072259


In [18]:
# Parallel (1 worker)

num_workers = 1

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=5,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=5, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)
del de

2020-08-07 00:58:01,730	INFO resource_spec.py:212 -- Starting Ray with 1.42 GiB memory available for workers and up to 0.73 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-07 00:58:01,989	WARNING services.py:923 -- Redis failed to start, retrying now.
2020-08-07 00:58:02,171	INFO services.py:1165 -- View the Ray dashboard at localhost:8265


Initializing and evaluating new population...


TypeError: missing a required argument: 'x'

In [ ]:
results = [de.f_objective.remote(de.population[i]) for i in range(de.pop_size)]

In [ ]:
# Parallel (2 workers)

num_workers = 2

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=5,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=5, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)

In [ ]:
# Parallel (4 workers)

num_workers = 4

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=5,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=5, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)

In [ ]:
# Parallel (20 workers)

num_workers = 20

start = time.time()
de = PDE(
    dimensions=len(bounds),
    configspace=False,            # if passing a custom search space and not ConfigSpace
    f=eggholder_objective_sleep,  # passing the objective function
    pop_size=5,                  # tunable hyperparameter (minimum limit determined by mutation strategy)
    mutation_factor=0.5,          # tunable hyperparameter (determines exploration)
    crossover_prob=0.5,           # tunable hyperparameter (determines exploitation)
    strategy='rand1_bin',         # tunable hyperparameter (trades-off exploration-exploitation)
    num_workers=num_workers
)
load_time = time.time() - start
start = time.time()
traj, runtime, hist = de.run(generations=5, verbose=True)
end = time.time() - start
    
print("Time to initialise client: {} seconds".format(load_time))
print("Time taken for parallel processing with {} workers: {} seconds".format(num_workers, end))
print(de.inc_config, de.inc_score)